In [2]:
import pandas as pd
import numpy as np

df = pd.read_stata("angrist.dta")
# drop columns what we generate later (e.g. education, wage) 
# and ones that are directly correlated with those columns (year of birth) or irrelevant (census)
df = df.drop(columns=['v2', 'v4', 'v8', 'v9', 'v16', 'v22', 'v27'])

# helper function to generate synthetic data
def generate_synthetic_data(df, n):
    synthetic_data = {}
    
    for col in df.columns:
        value_counts = df[col].value_counts(normalize=True) 
        values = value_counts.index.tolist()
        probabilities = value_counts.values

        synthetic_data[col] = np.random.choice(values, size=n, p=probabilities)
    
    return pd.DataFrame(synthetic_data)

synthetic_df = generate_synthetic_data(df, 100000)
synthetic_df.head()

,v1,v3,v5,v6,v7,v10,v11,v12,v13,v14,v15,v17,v18,v19,v20,v21,v23,v24,v25,v26
0,42,2,0,0,15,1,0,0,0,0,0,36,2,0,1,0,52,0,0,0
1,40,3,0,0,16,1,0,0,0,0,1,18,2,0,1,0,52,0,0,1
2,43,2,0,0,22,1,0,0,0,0,0,17,4,0,0,0,52,0,0,0
3,45,2,0,0,16,0,0,0,0,0,1,40,3,0,1,0,36,0,0,0
4,46,3,0,0,14,1,1,0,0,0,0,17,1,1,1,0,52,0,1,0


In [48]:
# verify the synthetic data is similar to original
summary = synthetic_df.describe().loc[['mean', 'std', 'min', 'max']]
print(summary)
summary = df.describe().loc[['mean', 'std', 'min', 'max']]
print(summary)

            v3        v5        v6         v7       v10       v11       v12  \
mean  1.880000  0.201770  0.063320  15.022690  0.843960  0.165250  0.048410   
std   0.565122  0.401323  0.243539   3.236044  0.362895  0.371408  0.214632   
min   0.000000  0.000000  0.000000   0.000000  0.000000  0.000000  0.000000   
max   3.000000  1.000000  1.000000  22.000000  1.000000  1.000000  1.000000   

           v13      v14       v15        v17       v19      v20       v21  \
mean  0.055870  0.00658  0.128780  30.775940  0.082340  0.21340  0.165300   
std   0.229672  0.08085  0.334958  14.664968  0.274883  0.40971  0.371453   
min   0.000000  0.00000  0.000000   1.000000  0.000000  0.00000  0.000000   
max   1.000000  1.00000  1.000000  99.000000  1.000000  1.00000  1.000000   

           v23       v24       v25       v26  
mean  38.63186  0.075390  0.094170  0.149830  
std   19.93551  0.264021  0.292067  0.365161  
min    0.00000  0.000000  0.000000 -1.000000  
max   52.00000  1.000000  1.00

In [ ]:
from sklearn.model_selection import train_test_split
import os

In [ ]:
# basic test set
N = 100000
filepath = './basic/'

u1 = np.random.normal(0, 1, N) # educ shock
v1 = np.random.normal(0, 1, N)  # wage shock
qob = np.random.randint(1, 5, size=N) 
age = np.random.randint(30, 51, size=N)

# straightforward, test set: ./basic/
educ = 10 + 1.5 * qob + u1
wage = 5 + 2.5 * educ + v1

df = pd.DataFrame({
    'y1': wage,        # Outcome: Wages
    'x1': age,        # Other variable: Schooling
    'z1': qob,        # Instrument
    't1' : educ         # Treatment
})

angrist = pd.read_stata("angrist.dta")
synth = generate_synthetic_data(angrist, N)
synth.columns = ['x' + str(i) for i in range(2, len(angrist.columns) + 2)]
combined_df = pd.concat([df, synthetic_df], axis=1)
if not os.path.isdir(filepath):
    os.makedirs(filepath)
train_df, temp_df = train_test_split(combined_df, test_size=0.3)
val_df, test_df = train_test_split(temp_df, test_size=0.5)
train_df.to_csv(filepath + "train.csv", index=False)
val_df.to_csv(filepath + "valid.csv", index=False)
test_df.to_csv(filepath + "test.csv", index=False)

In [80]:
# one weak instrument
N = 100000
filepath = './weak/'

u1 = np.random.normal(0, 1, N) # educ shock
v1 = np.random.normal(0, 1, N)  # wage shock
qob = np.random.randint(1, 5, size=N) 
age = np.random.randint(30, 51, size=N)

educ = 10 + 0.01 * qob + u1
wage = 5 + 2.5 * educ + v1

df = pd.DataFrame({
    'y1': wage,        # Outcome: Wages
    'x1': age,        # Other variable: Schooling
    'z1': qob,        # Instrument
    't1' : educ         # Treatment
})

angrist = pd.read_stata("angrist.dta")
synth = generate_synthetic_data(angrist, N)
synth.columns = ['x' + str(i) for i in range(2, len(angrist.columns) + 2)]
combined_df = pd.concat([df, synthetic_df], axis=1)
if not os.path.isdir(filepath):
    os.makedirs(filepath)
train_df, temp_df = train_test_split(combined_df, test_size=0.3)
val_df, test_df = train_test_split(temp_df, test_size=0.5)
train_df.to_csv(filepath + "train.csv", index=False)
val_df.to_csv(filepath + "valid.csv", index=False)
test_df.to_csv(filepath + "test.csv", index=False)

In [81]:
# strong but endogenous instrument
N = 100000
filepath = './endog/'

u1 = np.random.normal(0, 1, N) # educ shock
v1 = np.random.normal(0, 1, N)  # wage shock
qob = np.random.randint(1, 5, size=N) 
age = np.random.randint(30, 51, size=N)

educ = 10 + 1.5 * qob + u1
wage = 5 + 2.5 * educ + 0.5 * qob + v1

df = pd.DataFrame({
    'y1': wage,        # Outcome: Wages
    'x1': age,        # Other variable: Schooling
    'z1': qob,        # Instrument
    't1' : educ         # Treatment
})

angrist = pd.read_stata("angrist.dta")
synth = generate_synthetic_data(angrist, N)
synth.columns = ['x' + str(i) for i in range(2, len(angrist.columns) + 2)]
combined_df = pd.concat([df, synthetic_df], axis=1)
if not os.path.isdir(filepath):
    os.makedirs(filepath)
train_df, temp_df = train_test_split(combined_df, test_size=0.3)
val_df, test_df = train_test_split(temp_df, test_size=0.5)
train_df.to_csv(filepath + "train.csv", index=False)
val_df.to_csv(filepath + "valid.csv", index=False)
test_df.to_csv(filepath + "test.csv", index=False)

In [82]:
# one weak and endog
N = 100000
filepath = './weakendog/'

u1 = np.random.normal(0, 1, N) # educ shock
v1 = np.random.normal(0, 1, N)  # wage shock
qob = np.random.randint(1, 5, size=N) 
age = np.random.randint(30, 51, size=N)

educ = 10 + 0.01 * qob + u1
wage = 5 + 2.5 * educ + 0.5 * qob + v1

df = pd.DataFrame({
    'y1': wage,        # Outcome: Wages
    'x1': age,        # Other variable: Schooling
    'z1': qob,        # Instrument
    't1' : educ         # Treatment
})

angrist = pd.read_stata("angrist.dta")
synth = generate_synthetic_data(angrist, N)
synth.columns = ['x' + str(i) for i in range(2, len(angrist.columns) + 2)]
combined_df = pd.concat([df, synthetic_df], axis=1)
if not os.path.isdir(filepath):
    os.makedirs(filepath)
train_df, temp_df = train_test_split(combined_df, test_size=0.3)
val_df, test_df = train_test_split(temp_df, test_size=0.5)
train_df.to_csv(filepath + "train.csv", index=False)
val_df.to_csv(filepath + "valid.csv", index=False)
test_df.to_csv(filepath + "test.csv", index=False)

In [83]:
# many weak
N = 100000
filepath = './manyweak/'

angrist = pd.read_stata("angrist.dta")
synth = generate_synthetic_data(angrist, N)
synth.columns = ['x' + str(i) for i in range(2, len(angrist.columns) + 2)]

u1 = np.random.normal(0, 1, N) # educ shock
v1 = np.random.normal(0, 1, N)  # wage shock
qob = np.random.randint(1, 5, size=N) 
age = np.random.randint(30, 51, size=N)

df = pd.DataFrame({
    'u1': u1,    
    'v1': v1,   
    'x1': age,       
    'z1' : qob         
})

combined_df = pd.concat([df, synthetic_df], axis=1)
combined_df['const'] = 1

combined_df.head()


weights = {'v1': 0, 'const': 10, 'u1': 1}  
for col in combined_df.columns:
    if col not in weights:
        weights[col] = 0.01

selected_columns = list(weights.keys())

t1 = np.array([
    sum(row[col] * weights[col] for col in selected_columns)
    for _, row in combined_df.iterrows()
])

t1 = t1[:,0]
combined_df['t1'] = t1

# wage = 5 + 1.5 educ + v1
weights = { 'const': 5, 'v1': 1, 't1': 1.5}  
selected_columns = ['const', 'v1', 't1']
y1 = np.array([
    sum(row[col] * weights[col] for col in selected_columns)
    for _, row in combined_df.iterrows()
])

y1 = y1[:,0]
combined_df['y1'] = y1

if not os.path.isdir(filepath):
    os.makedirs(filepath)
train_df, temp_df = train_test_split(combined_df, test_size=0.3)
val_df, test_df = train_test_split(temp_df, test_size=0.5)
train_df.to_csv(filepath + "train.csv", index=False)
val_df.to_csv(filepath + "valid.csv", index=False)
test_df.to_csv(filepath + "test.csv", index=False)

In [ ]:
# many weak, one strong
N = 100000
filepath = './manyweak1strong/'

angrist = pd.read_stata("angrist.dta")
synth = generate_synthetic_data(angrist, N)
synth.columns = ['x' + str(i) for i in range(2, len(angrist.columns) + 2)]

u1 = np.random.normal(0, 1, N) # educ shock
v1 = np.random.normal(0, 1, N)  # wage shock
qob = np.random.randint(1, 5, size=N) 
age = np.random.randint(30, 51, size=N)

df = pd.DataFrame({
    'u1': u1,    
    'v1': v1,   
    'x1': age,       
    'z1' : qob         
})

combined_df = pd.concat([df, synthetic_df], axis=1)
combined_df['const'] = 1

combined_df.head()


weights = {'z1': 1.5, 'v1': 0, 'const': 10, 'u1': 1}  
for col in combined_df.columns:
    if col not in weights:
        weights[col] = 0.01

selected_columns = list(weights.keys())

t1 = np.array([
    sum(row[col] * weights[col] for col in selected_columns)
    for _, row in combined_df.iterrows()
])

t1 = t1[:,0]
combined_df['t1'] = t1

# wage = 5 + 1.5 educ + v1
weights = { 'const': 5, 'v1': 1, 't1': 1.5}  
selected_columns = ['const', 'v1', 't1']
y1 = np.array([
    sum(row[col] * weights[col] for col in selected_columns)
    for _, row in combined_df.iterrows()
])

y1 = y1[:,0]
combined_df['y1'] = y1

if not os.path.isdir(filepath):
    os.makedirs(filepath)
train_df, temp_df = train_test_split(combined_df, test_size=0.3)
val_df, test_df = train_test_split(temp_df, test_size=0.5)
train_df.to_csv(filepath + "train.csv", index=False)
val_df.to_csv(filepath + "valid.csv", index=False)
test_df.to_csv(filepath + "test.csv", index=False)

In [27]:
data = CausalDataset('./endog')

model = Vanilla2SLS()
model.fit(data)
ITE = model.predict(data.train)
ATE,_ = model.ATE(data.train)

ATE

Run -1-th experiment for Vanilla2SLS. 
End. --------------------


2.832858044676648

In [ ]:
### vanilla 2sls works, simple model?
u1 = np.random.normal(0, 1, N) # educ shock
v1 = np.random.normal(0, 1, N)  # wage shock
qob = np.random.randint(1, 5, size=N) 
age = np.random.randint(30, 41, size=N)

educ = 10 + 5 * qob + u1
wage = 5 + 3.5 * educ + v1

# weak instrument test, vanilla 2sls doesn't work
educ = 10 + 0.01 * qob + u1
wage = 5 + 3.5 * educ + v1

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Set seed for reproducibility
# np.random.seed(42)

# Simulate data
N = 100000  # Number of observations

# maybe this works? vanilla does not see the effect of compulsory schooling on wages
u1 = np.random.normal(0, 1, N) # educ shock
v1 = np.random.normal(0, 1, N)  # wage shock
qob = np.random.randint(1, 5, size=N) 
age = np.random.randint(30, 41, size=N)


educ = 10 + 0.01 * qob + u1
wage = 5 + 3.5 * educ + v1

# Create DataFrame with labeled columns
df = pd.DataFrame({
    'y1': wage,        # Outcome: Wages
    'x1': age,        # Other variable: Schooling
    'z1': qob,        # Instrument
    't1': educ         # Treatment
})

# Save DataFrame to CSV
df.to_csv('generated_data.csv', index=False)

df = pd.read_csv("generated_data.csv")
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
train_df.to_csv("train.csv", index=False)
val_df.to_csv("valid.csv", index=False)
test_df.to_csv("test.csv", index=False)

data = CausalDataset('./')

model = Vanilla2SLS()
model.fit(data)
ITE = model.predict(data.train)
ATE,_ = model.ATE(data.train)

ATE


Run -1-th experiment for Vanilla2SLS. 
End. --------------------


3.1659505273539454

In [5]:
from mliv.inference import Vanilla2SLS
from mliv.utils import CausalDataset
from mliv.inference import Poly2SLS

In [ ]:

def genData(filepath, N=100000):
    u1 = np.random.normal(0, 1, N) # educ shock
    v1 = np.random.normal(0, 1, N)  # wage shock
    qob = np.random.randint(1, 5, size=N) 
    age = np.random.randint(30, 51, size=N)
    
    # straightforward, test set: ./basic/
    # educ = 10 + 1.5 * qob + u1
    # wage = 5 + 2.5 * educ + v1

    # weak instrument, ./weak/
    educ = 10 + 0.01 * qob + u1
    wage = 5 + 2.5 * educ + v1
    
    # Create DataFrame with labeled columns
    df = pd.DataFrame({
        'y1': wage,        # Outcome: Wages
        'x1': age,        # Other variable: Schooling
        'z1': qob,        # Instrument
        't1' : educ         # Treatment
    })
    
    angrist = pd.read_stata("angrist.dta")
    synth = generate_synthetic_data(angrist, N)
    synth.columns = ['x' + str(i) for i in range(2, len(angrist.columns) + 2)]
    combined_df = pd.concat([df, synthetic_df], axis=1)
    if not os.path.isdir(filepath):
        os.makedirs(filepath)
    train_df, temp_df = train_test_split(combined_df, test_size=0.3)
    val_df, test_df = train_test_split(temp_df, test_size=0.5)
    train_df.to_csv(filepath + "train.csv", index=False)
    val_df.to_csv(filepath + "valid.csv", index=False)
    test_df.to_csv(filepath + "test.csv", index=False)

In [16]:
from sklearn.model_selection import train_test_split
df = pd.read_csv("generated_data.csv")
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
train_df.to_csv("train.csv", index=False)
val_df.to_csv("valid.csv", index=False)
test_df.to_csv("test.csv", index=False)

data = CausalDataset('./')

model = Vanilla2SLS()
model.fit(data)
ITE = model.predict(data.train)
ATE,_ = model.ATE(data.train)

ATE

Run -1-th experiment for Vanilla2SLS. 
End. --------------------


3.1659505273539454

In [ ]:
### vanilla 2sls works, simple model?
u1 = np.random.normal(0, 1, N) # educ shock
v1 = np.random.normal(0, 1, N)  # wage shock
qob = np.random.randint(1, 5, size=N) 
age = np.random.randint(30, 41, size=N)

educ = 10 + 5 * qob + u1
wage = 5 + 3.5 * educ + v1

# weak instrument test, vanilla 2sls doesn't work
educ = 10 + 0.01 * qob + u1
wage = 5 + 3.5 * educ + v1

In [55]:
epsilon = np.random.normal(0, 1, N)  # wage shock
nu = np.random.normal(0, 1, N) # educ shock

# x1 = np.random.normal(0, 1, N) # age

t1 = np.random.binomial(10, 0.5, N)  # compulsory
z1 = np.random.normal(0, 1, N) # qob
e = np.random.normal(0, 1, N) 

In [ ]:
# gives nothing
x1 = e + 1.5 * t1 + 0.5 * z1  #educ
y1 = 10 + 3 * x1

# gives 1.5
x1 = 3 + 1.5 * t1 + 0.5 * z1  #educ
y1 = 10 + 3 + 1.5 * t1


# Create DataFrame with labeled columns
df = pd.DataFrame({
    'y1': y1,        # Outcome: Wages
    'x1': x1,        # Other variable: Schooling
    'z1': z1,        # Instrument
    't1': t1         # Treatment
})

# Save DataFrame to CSV
df.to_csv('generated_data.csv', index=False)
print("Data saved to 'generated_data.csv'")

# ------------------------- 2SLS Implementation -------------------------
# Stage 1: Regress treatment (t1) on instrument (z1)
X_first_stage = np.column_stack((z1, t1))  # IV and Treatment as predictors
first_stage_model = LinearRegression()
first_stage_model.fit(X_first_stage, x1)
x1_hat = first_stage_model.predict(X_first_stage)  # Predicted schooling

# Stage 2: Regress outcome (y1) on predicted schooling (x1_hat) and treatment (t1)
X_second_stage = np.column_stack((x1_hat, t1))  # Predicted schooling + Treatment
second_stage_model = LinearRegression()
second_stage_model.fit(X_second_stage, y1)
beta_s_2sls = second_stage_model.coef_[0]  # Effect of schooling on wages (from 2SLS)
beta_t_2sls = second_stage_model.coef_[1]  # Effect of treatment on wages (from 2SLS)

# Print 2SLS estimates
print(f"Estimated effect of schooling (2SLS): {beta_s_2sls:.4f}")
print(f"Estimated effect of treatment (2SLS): {beta_t_2sls:.4f}")

Data saved to 'generated_data.csv'
Estimated effect of schooling (2SLS): -0.0000
Estimated effect of treatment (2SLS): 1.5000
